
# Project: Investigate Student grads

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
### Introduction

> **into**: we have here student-dataset consistences of 1000 rows and 8 columns  
the main features:
['gender', 'race/ethnicity', 'parental level of education', 'lunch','test preparation course', 'math score', 'reading score','writing score']

>**what are we trying to know ?**

<ul>
 
    
    
   <li><a href="#Q1">Q1</a></li>
    who has the more probaility to sucess for " each category " ?
    
   <li><a href="#Q3">Q2</a></li>
    compareing between the 3 subjects
    
   <li><a href="#Q2">Q3</a></li>
    which features effect more about student success ? 
    
    
   <li><a href="#Q4">Q4</a></li>
    T-tests "
    
 </ul>


In [ ]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy import stats

In [ ]:
raw_data=pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')
raw_data.head()

In [ ]:
raw_data.isnull().any()

In [ ]:
df=raw_data.copy()

#cluclating the avg 
df['avg']=round((df['math score'] + df['reading score']+df['writing score'])/3,2)

#passing or not 
df['pass']=(df['avg'] > 60 ) *1

# <a id='eda'></a>
### Exploratory Data Analysis



In [ ]:
print('the passing overall rate is :' ,df['pass'].mean())

In [ ]:
print('the passing rate in females is :' ,round(df.query('gender=="female"')['pass'].mean(),2))

In [ ]:
print('the passing rate in males is :' ,round(df.query('gender=="male"')['pass'].mean(),2))

<a id='Q1'></a>
#### who has the more probaility to sucess for " each category " ?

In [ ]:
#a function to get the passing proportion :  

def get_prob(col):
    df[df['pass'] == 1].groupby([col]).count()[['pass']]
    df.groupby([col]).count()[['avg']]
    result=pd.concat([df[df['pass'] == 1].groupby([col]).count()[['pass']],df.groupby([col]).count()[['avg']]], axis=1)
    result['prob'] =result['pass'] / result['avg']
    sns.set(rc={'figure.figsize':(5,5)})
    plt.title(col)
    return result['prob'].plot(kind='barh') , result[['prob']]
    
    


In [ ]:
get_prob('gender');

* the female success more than the male 

In [ ]:
get_prob('race/ethnicity');

* the Group E success more than any group " the asian people of course :D" 

In [ ]:
get_prob('parental level of education');

* bachelor and master parents holder have the more sucess kids

In [ ]:
get_prob('lunch');

* students that have lunch have sucess more than stuednt who don't , has something to do with displin I think

In [ ]:
get_prob('test preparation course');

* the completed preparation course students have sucess more than stuednt who don't , 

--------------------------------------------------------------------------------------------------------------------

# <a id='Q3'></a>

#### compareing between the 3 subjects


In [ ]:
sns.set(rc={'figure.figsize':(20,7)})


In [ ]:
plt.subplot(1, 2, 2);
sns.histplot(data=df,x='math score',fill=False,color='r', element="step");

plt.subplot(1, 2, 2);
sns.histplot(data=df,x='writing score',fill=False,color='y', element="step");

plt.subplot(1, 2, 2);
sns.histplot(data=df,x='reading score',fill=False,color='b', element="step");
plt.title('comperaing subject');


print('The white is reading , the writing is Yellow');
raw_data.describe()

In [ ]:
x=df.iloc[:, 0:5].apply(LabelEncoder().fit_transform)
y=df[['avg']]

 * **the reading** and **wiriting** is kinda same which says it corrlated , otherwise **the math** grades is skweded to the right which suggest that the **math** grades is more litter 

<a id='Q2'></a>
#### which features effect more about student success ?

In [ ]:
sns.set(rc={'figure.figsize':(7,7)})

#using ANOVA in order to determine which feature effect the passing

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


model = SelectKBest(score_func=f_classif, k=4)
fit=model.fit(x,y)

feat_importances = pd.Series(fit.scores_, index=x.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.title('which feature effects to pass')

* as we see that the **lunch** effects the most 

### we are going to run T-tests in order to make sure which cateogry is better 

-------------------------------------------------------------------------------------------------------------------------- 

<a id='Q4'></a>
### T-test

In [ ]:
g = sns.FacetGrid(df, col="lunch",sharey=False)
g.fig.suptitle('compare between having a lunch or not grades');
g.map_dataframe(plt.hist,x='avg',data=df);
g.set_titles(col_template="{col_name}", row_template="{row_name}");

In [ ]:
rv1=df.query('lunch == "standard"')['avg']
rv2=df.query('lunch != "standard"')['avg']
stats.ttest_ind(rv1,rv2,equal_var=False)

 * we have a **p-value** smaller than 0.05 which makes us accept that there's a big differance between between the students who takes the lunch and the others who don't 

In [ ]:
g = sns.FacetGrid(df, col="test preparation course",sharey=False)
g.fig.suptitle('test preparation course');
g.map_dataframe(plt.hist,x='avg',data=df);
g.set_titles(col_template="{col_name}", row_template="{row_name}");

In [ ]:
rv1=df[df['test preparation course']=='none'][['avg']]
rv2=df[df['test preparation course']!='none'][['avg']]
stats.ttest_ind(rv1,rv2,equal_var=False)

* we have a **p-value** smaller than 0.05 which makes us accept that there's a big differance between the students who didn't take the preparation course better than others who did , 

In [ ]:
g = sns.FacetGrid(df, col="gender",sharey=False)
g.fig.suptitle('gender');
g.map_dataframe(plt.hist,x='avg',data=df);
g.set_titles(col_template="{col_name}", row_template="{row_name}");

In [ ]:
rv1=df[df['gender']=='male'][['avg']]
rv2=df[df['gender']!='male'][['avg']]
stats.ttest_ind(rv1,rv2,equal_var=False)

* we have a **p-value** smaller than 0.05 ,which makes us accept that there's a big differance between male and femlaes , and by looking to the **t statistic** it says that   **females** better than **males** . 

<a id='conclusions'></a>
## Conclusions

* the most effective feature is takeing the lucnch, and the student who take the luch have more probiblty to be suecceful than the students who don't. 

* the student who didn't take the prepartion course is more successful than the studenst who didn't. 

* females students  is more successful than males